In [2]:
! ls

02514-Deep-Learning-In-Computer-Vision	DLCI-venv  Desktop


In [3]:
import os
os.chdir('02514-Deep-Learning-In-Computer-Vision/hotdog/')

In [4]:
from dataloader import *
import torch.nn.functional as F
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm
import torch.nn as nn


In [5]:
data_train = HotdogDataset()
data_test = HotdogDataset()

In [6]:
loader = next(iter(data_train.get_dataloader(batch_size=21)))


In [7]:
data_test.get_dataloader()[0]

In [8]:
len(data_train)

2047

In [9]:
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The code will run on GPU.


In [10]:
class PeetzCNN_threee(nn.Module):
    def __init__(self, dropout = 0.5, batchnorm = True):
        super(PeetzCNN_threee, self).__init__()
        
        self.convolutional = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        
        self.batchnorm_layer = nn.Sequential(nn.BatchNorm2d(128),) if batchnorm else nn.Sequential()

   
        self.fully_connected = nn.Sequential(
            nn.Linear(32*32*128, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256,100),
            nn.ReLU(),
            nn.Linear(100,2),
            nn.LogSigmoid()
        )


    def forward(self, x):
        x = self.convolutional(x)
        x = self.batchnorm_layer(x)
        #reshape x so it becomes flat, except for the first dimension (which is the minibatch)
        x = x.view(x.size(0), -1)
        x = self.fully_connected(x)
        return x

In [11]:
model = PeetzCNN_threee()
model.to(device)
#Initialize the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# Expand data
train_loader = data_train.get_dataloader()[0]
test_loader = data_test.get_dataloader()[1]
#Get the first minibatch
data = next(iter(train_loader))[0].cuda()
#Try running the model on a minibatch
print('Shape of the output from the convolutional part', model.convolutional(data).shape)
#model(data); #if this runs the model dimensions fit

Shape of the output from the convolutional part torch.Size([64, 128, 32, 32])


In [53]:


num_epochs = 5

for epoch in tqdm(range(num_epochs), unit='epoch'):
    #For each epoch
    train_correct = 0
    model.train()
    for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device)
        #Zero the gradients computed for each weight
        optimizer.zero_grad()
        #Forward pass your image through the network
        output = model(data)
        #Compute the loss
        loss = F.nll_loss(torch.log(output), target)
        #Backward pass through the network
        loss.backward()
        #Update the weights
        optimizer.step()
        
        #Compute how many were correctly classified
        predicted = output.argmax(1)
        train_correct += (target==predicted).sum().cpu().item()
    #Comput the test accuracy
    test_correct = 0
    model.eval()
    for data, target in test_loader:
        data = data.to(device)
        with torch.no_grad():
            output = model(data)
        predicted = output.argmax(1).cpu()
        test_correct += (target==predicted).sum().item()
    train_acc = train_correct/len(data_train)
    test_acc = test_correct/len(data_test)
    print("Accuracy train: {train:.1f}%\t test: {test:.1f}%".format(test=100*test_acc, train=100*train_acc))
     


  0%|          | 0/5 [00:00<?, ?epoch/s]

  0%|          | 0/64 [00:00<?, ?it/s]

Accuracy train: 51.8%	 test: 48.1%


  0%|          | 0/64 [00:00<?, ?it/s]

Accuracy train: 52.5%	 test: 48.1%


  0%|          | 0/64 [00:00<?, ?it/s]

Accuracy train: 52.5%	 test: 48.1%


  0%|          | 0/64 [00:00<?, ?it/s]